In [ ]:
import os
import shutil
import numpy as np
import cv2

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

root = './'

file_paths = [
    root + 'training10_0.tfrecords',
    root + 'training10_1.tfrecords',
    root + 'training10_2.tfrecords',
    root + 'training10_3.tfrecords',
    root + 'training10_4.tfrecords',
]

class TFRecordExtractor:
    def __init__(self, file_paths):
        self.file_paths = [os.path.abspath(file_path) for file_path in file_paths]
        self.count = 0
        
    def get_images(self):
        dataset = tf.data.TFRecordDataset(self.file_paths)
        dataset = dataset.map(self._extract_fn)
        iterator = dataset.make_one_shot_iterator()
        next_element = iterator.get_next()
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            image_data_list = []
            try:
                while True:
                    image_data = sess.run(next_element)
                    image_data_list.append(image_data)
            except:
                pass
            
            return image_data_list
        
    def _extract_fn(self, tfrecord):
        # Extract features using keys
        features = {
            'label': tf.FixedLenFeature([], tf.int64),
            'label_normal': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string)
        }
        
        parsed_data = tf.parse_single_example(tfrecord, features=features)
        label = parsed_data['label']
        image = tf.decode_raw(parsed_data['image'], tf.uint8)
        image = tf.reshape(image, [299, 299, 1])
        
        return [image, label]
    
    def post_process_images(self):
        image_data_list = self.get_images()
        benign_count = 0
        malignant_count = 0
        total_count = 0
        
        type_dict = {
            0: 'normal',
            1: 'benign_calci',
            2: 'benign_mass',
            3: 'malignant_calci',
            4: 'malignant_mass',
        }
        
        for image_data in image_data_list:
            label = image_data[1]
                
            if label == 0 or label == 1 or label == 3:
                continue
                
            total_count += 1
            
            if label == 2:
                benign_count += 1
            else:
                malignant_count += 1
            
            file_name = 'images/train/' + type_dict[label] + \
                        '/' + type_dict[label] + '_' + str(total_count) + '.png'
            
            img = image_data[0]
            cv2.imwrite(root + file_name, img)
            
            with open(root + 'images/train/train.txt', 'a') as f:
                f.write(file_name + ' ' + ('0' if label == 2 else '1') + '\n')
        
        print('total: ' + str(total_couut))
        print('benign: ' + str(benign_count))
        print('malignant: ' + str(malignant_count))

TFRecordExtractor(file_paths).post_process_images()

In [ ]:
# Load test set

test_data = np.load('./test10_data.npy')
test_label = np.load('./test10_labels.npy')

total_count = 0

type_dict = {
            2: 'benign_mass',
            4: 'malignant_mass',
}

for i in range(len(test_label)):
    label = test_label[i]
    image = test_data[i]
    
    if not (label == 2 or label == 4):
        continue
        
    total_count += 1
    
    file_name = 'images/test/' + type_dict[label] + '/' + type_dict[label] + '_' + str(total_count) + '.png'
#     cv2.imwrite(file_name, image)

    with open(root + 'images/test/test.txt', 'a+') as f:
        f.write(file_name + ' ' + ('0' if label == 2 else '1') + '\n')